In [1]:
from model import Encoder, Decoder, Seq2Seq
from data_loader import *
import pandas as pd
import torch.optim.lr_scheduler as lr_scheduler
from torch import optim
import torch.nn.functional as F
import datetime
import pretty_midi
import glob

In [2]:
import os
import matplotlib
import math
matplotlib.use('Agg')
# matplotlib.use("QtAgg")
import ffmpeg
#conda install -c conda-forge ffmpeg-python

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
plt.rcParams['animation.ffmpeg_path'] = '/home/ilc/anaconda3/bin/ffmpeg'#'/usr/bin/ffmpeg'

import numpy as np
import subprocess as sp
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip

from midi2audio import FluidSynth

from torch.autograd import Variable 

In [3]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
dataset_name_path = f"./midi_list.txt"
dataloader = get_dataloader(dataset_name_path, batch_size=128) #[20, 512, 128], [20, 512, 102]

val_dataset_name_path = f"./midi_list_eval.txt"
val_dataloader = get_val_dataloader(val_dataset_name_path, batch_size=128) #[20, 512, 128], [20, 512, 102]

learning_rate = 0.001#0.001

input_size_encoder = 128 #129 #128
input_size_decoder = 102 #102 #24
output_size = 102#102 #24

# encoder_embedding_size = 300
# decoder_embedding_size = 300
enc_dropout = 0.5
dec_dropout = 0.
step = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

self.piece_count:  105
dataset_len:  10500
val_dataset_len 5
cuda:0


In [5]:
class LSTM1(nn.Module):
    def __init__(self, output_dim, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.output_dim = output_dim #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, output_dim) #fully connected to determine output dim

        self.relu = nn.ReLU()

    def forward(self,x):
        # h0, c0 no time information
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
        # Propagate input through LSTM
        # x is MIDI => [44, 512, 128]

        # hn is final state, run over the sequence length
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        # hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        # print("output.shape", output.shape)
        # print("hn.shape", hn.shape)
        # out = self.relu(hn)
        out = self.fc_1(output) #final
        return out
 

In [6]:
# Define the model architecture
input_size = 128 #number of features
hidden_size = 1024 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
seq_len = 512
output_dim = 102 #number of output classes

# model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
# model = LSTM(embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
model = LSTM1(output_dim, input_size, hidden_size, num_layers, seq_len).to(device) #our lstm class
model.train()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

num_epochs = 100 #10
avg_loss_list = []
all_loss_list = []
val_loss_per_epoch_list = []

In [7]:
def customized_mse_loss(output, target, prev_output, midi_array):
    # target = target.transpose(0, 1)
    # print("output", output)
    # print(output.shape) #torch.Size([20, 513, 102])
    # print(target.shape) #torch.Size([20, 513, 102])
    mse_loss = F.mse_loss(output, target)
    # print("mse_loss:", mse_loss)

    var_diff = torch.var(torch.squeeze(output), dim=1, keepdim=True)
    mean_diff = torch.mean(var_diff)
    
    # Condition 1: Penalize if output is similar to previous output
    if mean_diff < 1e-4: #threshold
        #output [512, 1, 102] => [102] <-> [102] <-> [102] <-> ... <-> [102]
        mse_loss *= 1000

    # Condition 2: Stop movement if input is all zeros
    # midi_transpose = midi_array.transpose(0, 1)
    # midi_sum_row = torch.sum(midi_transpose, dim=-1)
    # mask = midi_sum_row == 0
    # mask = mask.unsqueeze(-1)
    # mask = mask.to(device)
    # # according to recorded index, make a mask [0, 1, 1, 0, ..., 0], true part will be omit(set value to 0).
    # # before compute mse, use mask first to tensor, then caculate MES loss
    # masked_output = output.masked_fill(mask, 0) #inplace function
    # masked_target = target.masked_fill(mask, 0)
    # mse_loss += F.mse_loss(masked_output, masked_target) * 100 #output 和 previous output 不像的話，增大 loss

    # Condition 3: Penalize if right-hand movement is too different between outputs
    # if output.shape[-1] == 21:  # Assumes hand joints are the last 21 dimensions
    #     rh_indices = [i for i in range(12, 21)]  # Right-hand joint indices
    #     rh_output = output[..., rh_indices]
    #     rh_prev_output = prev_output[..., rh_indices]
    #     rh_loss = nn.functional.mse_loss(rh_output, rh_prev_output)
    #     if rh_loss > 0.1:
    #         mse_loss *= 1000

    return mse_loss

In [8]:
def evaluate_lstm(model, val_dataloader):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    counter = 0
    # previous_output = torch.zeros(512, 102).to(device)
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(val_dataloader): #tqdm(enumerate(val_dataloader), total=len(val_dataloader))
            counter += 1

            inputs = inputs.to(device).float()
            targets = targets.to(device).float()
            print("inputs.shape:", inputs.shape)
            print("targets.shape:", targets.shape)
            outputs = model(inputs)
            print("outputs.shape:", outputs.shape)

            loss =  F.mse_loss(outputs, targets)
            valid_running_loss += loss.cpu().item()
            # previous_output = outputs

    epoch_val_loss = valid_running_loss / counter
    return epoch_val_loss

In [9]:
# train the model
for epoch in range(num_epochs):
    # previous_output = torch.zeros(1, 512, 102).to(device)
    losses = []
    for i, (midi_batch, motion_batch) in enumerate(dataloader):
        model.train()
        
        midi_batch = midi_batch.to(device).float()
        motion_batch = motion_batch.to(device).float()
        # print("midi_batch", midi_batch.shape)
        # print("motion_batch", motion_batch.shape)

        optimizer.zero_grad()
        output = model(midi_batch) #midi_batch
        # print("output.shape", output.shape)

        # motion_ground_truth_padding = F.pad(motion_batch, (0,0,0,1), value = 1) #<eot>
        
        # loss =  F.mse_loss(output, motion_ground_truth_padding)
        loss =  F.mse_loss(output, motion_batch)
        # loss = customized_mse_loss(output, motion_ground_truth_padding, previous_output, midi_batch)
        # loss = customized_mse_loss(output, motion_batch, previous_output, midi_batch)

        # losses 累計lose
        losses.append(loss.cpu().item())
        all_loss_list.append(loss.cpu().item())
        loss.backward()

        optimizer.step()
        mean_loss = sum(losses)/len(losses)

        print(f"Epoch {epoch}, batch {i}: loss = {loss.cpu().item():.6f}")

        # scheduler.step(1)
        # previous_output = output

        loc_dt = datetime.datetime.today()
        loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")

    val_loss = evaluate_lstm(model, val_dataloader) #CUDA out of memory
    val_loss_per_epoch_list.append(val_loss)
    print(f"Epoch {epoch}: val_loss = {val_loss:.6f}")
    # save_best_model(
    #         val_loss, epoch, model, optimizer, loss, loc_dt_format, mean_loss
    #     )
    avg_loss_list.append(mean_loss)
    loc_dt = datetime.datetime.today()
    loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")
    if (epoch+1)%100 == 0:
        torch.save({
            'epoch':epoch,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict(),
            'loss':loss
        }, "./model_save/[100epoch]LSTM_save_epoch_" + str(epoch)+ "_"+ str(loc_dt_format) + "_avg_loss_" + str(mean_loss) +".tar")

Epoch 0, batch 0: loss = 0.176477
Epoch 0, batch 1: loss = 0.160604
Epoch 0, batch 2: loss = 0.132114
Epoch 0, batch 3: loss = 0.909421
Epoch 0, batch 4: loss = 0.108143
Epoch 0, batch 5: loss = 0.130589
Epoch 0, batch 6: loss = 0.139260
Epoch 0, batch 7: loss = 0.143146
Epoch 0, batch 8: loss = 0.143223
Epoch 0, batch 9: loss = 0.139436
Epoch 0, batch 10: loss = 0.134850
Epoch 0, batch 11: loss = 0.126121
Epoch 0, batch 12: loss = 0.107145
Epoch 0, batch 13: loss = 0.070911
Epoch 0, batch 14: loss = 0.026751
Epoch 0, batch 15: loss = 0.012657
Epoch 0, batch 16: loss = 0.026397
Epoch 0, batch 17: loss = 0.019713
Epoch 0, batch 18: loss = 0.009768
Epoch 0, batch 19: loss = 0.006509
Epoch 0, batch 20: loss = 0.007342
Epoch 0, batch 21: loss = 0.009430
Epoch 0, batch 22: loss = 0.010369
Epoch 0, batch 23: loss = 0.010018
Epoch 0, batch 24: loss = 0.008882
Epoch 0, batch 25: loss = 0.007470
Epoch 0, batch 26: loss = 0.005829
Epoch 0, batch 27: loss = 0.004977
Epoch 0, batch 28: loss = 0.00

In [ ]:
print(loc_dt_format)
print(avg_loss_list)

2023-07-03_18-26-56
[0.012661974706687033, 0.0015919805918820202, 0.0014384243646636606, 0.0013333117822185159, 0.0012704872731119394, 0.0011976969931274653, 0.0011586979227140547, 0.001138485220260918, 0.0011270876505877824, 0.0011238000295124948, 0.0011117435079067946, 0.0010819684315938503, 0.001058813000563532, 0.0010297775475773961, 0.0009794084406457842, 0.0009079097653739154, 0.0008526342385448515, 0.0007832597584929317, 0.000732838009018451, 0.0006872653043828905, 0.0006302889150101691, 0.0005898956164019182, 0.0005345656567951664, 0.000579691807506606, 0.00048752442689146844, 0.0004499564801808447, 0.00040553833055309953, 0.0003762234018649906, 0.00035402149125002324, 0.0003553281488129869, 0.0003112580869928934, 0.00028985252685379237, 0.0002757128956145607, 0.0002683243575738743, 0.00024743842921452595, 0.00022837740532122552, 0.00022319893247913568, 0.0002167039849446155, 0.0002062338836840354, 0.00020026504999259487, 0.00019286284013651312, 0.00018520980811445042, 0.000184

In [ ]:
# def lr_lambda(epoch):
#     # LR to be 0.1 * (1/1+0.01*epoch)
#     base_lr = 0.1
#     factor = 0.01
#     return base_lr/(1+factor*epoch)

In [ ]:
# scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.3, total_iters=10)
# scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [ ]:
plt.cla()
plt.clf()

In [ ]:
print(len(avg_loss_list))
avg_loss_list_dataframe = pd.DataFrame(avg_loss_list)

100


In [ ]:
avg_loss_list_dataframe

,0
0,0.012662
1,0.001592
2,0.001438
3,0.001333
4,0.001270
...,...
95,0.000100
96,0.000093
97,0.000100
98,0.000090


In [ ]:
plt.plot(np.array(avg_loss_list_dataframe.index), np.array(avg_loss_list_dataframe[0]))
plt.savefig("avg_loss_training.jpg")
plt.show()

In [ ]:
loss_list_dataframe = pd.DataFrame(all_loss_list)

In [ ]:
plt.plot(np.array(loss_list_dataframe.index), np.array(loss_list_dataframe[0]))
plt.savefig("training_loss.jpg")
plt.show()

In [ ]:
def predict(model, input, target, device):
    model.eval()
    with torch.no_grad():
        input = torch.as_tensor(input).to(torch.float32).to(device)
        print(target.shape)
        target = torch.as_tensor(target).to(torch.float32).to(device)
        # TODO: target should be <sos>, should not random
        outputs = model(input)
        return outputs.cpu().numpy()

In [ ]:
def read_midi(filename, specific_fps):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(filename)

    piano_roll = midi_data.get_piano_roll(fs=specific_fps)  # 40fps #250fps
    piano_roll[piano_roll > 0] = 1

    return piano_roll

In [ ]:
test_datapath = "./BWV1001/"
change_fps = 40
test_midi_path_list = glob.glob(test_datapath + "*.mid")
test_data_list = []
test_music_list = []
for test_midi in test_midi_path_list:
    str_name = test_midi
    print("str_name:", str_name)
    filename = str_name.split('/')[2]
    filecode = filename.split('.')[0]
    print("filecode: ",filecode)
    test_music_list.append(filecode)
    
    print(test_midi)
    read_piano_roll = read_midi(test_midi, change_fps)
    read_piano_roll_transpose = read_piano_roll.T
    print(read_piano_roll_transpose.shape)
    test_midi_len = read_piano_roll_transpose.shape[0]
    test_data_list.append(read_piano_roll_transpose)

str_name: ./BWV1001/vs1-1ada.mid
filecode:  vs1-1ada
./BWV1001/vs1-1ada.mid
(8171, 128)
str_name: ./BWV1001/vs1-2fug.mid
filecode:  vs1-2fug
./BWV1001/vs1-2fug.mid
(11537, 128)
str_name: ./BWV1001/vs1-3sic.mid
filecode:  vs1-3sic
./BWV1001/vs1-3sic.mid
(6993, 128)
str_name: ./BWV1001/vs1-4prs.mid
filecode:  vs1-4prs
./BWV1001/vs1-4prs.mid
(7897, 128)


In [ ]:
def column(matrix, i):
    return [row[i] for row in matrix]

def test_render_animation(fps, output, azim, prediction, ground_truth=None):
    prediction_array = np.asarray(prediction)
    print(prediction_array.size)
    limit = len(prediction_array)
    print("limit", limit)
    size = 6#6
    fps = 40

    # Skeleton layout
    parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                [8, 18], [8, 20], [13, 21], [13, 19],
                [5, 20], [5, 21], [7, 18], [7, 19],
                [18, 19], [19, 21], [21, 20], [20, 18], #waist
                [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
    # joints_right = [1, 2, 12, 13, 14]

    prediction_array[:, :, 2] += 0.1 #[:, :, 2]
    if ground_truth is not None:
        ground_truth[:, :, 2] += 0.1
        poses = {'Prediction': prediction_array,
                 'Ground_truth': ground_truth}
    else:
        poses = {'Prediction': prediction_array}
    

    fig = plt.figure()#(figsize=(size*len(poses), size))
    # ax_3d = []
    # lines_3d = []
    radius = 1#14 #3.7#
    # print(poses)
    for index, (title, data) in enumerate(poses.items()):
        ax = fig.add_subplot(1, len(poses), index + 1, projection='3d')
        ax.clear()
        print(data)
        ims = [] #每一 frame 都存
        for frame_index, each_frame in enumerate(data):
            # print("each_frame")
            # print(each_frame)
            ax.view_init(elev=15., azim=azim)
            ax.set_xlim3d([-radius/2, radius/2])
            ax.set_zlim3d([0, radius])
            ax.set_ylim3d([-radius/2, radius/2])
            ax.set_aspect('auto') #ax.set_aspect('equal')

            # print(title)
            points = ax.scatter(column(each_frame[:30], 0), column(each_frame[:30], 1), column(each_frame[:30], 2), cmap='jet', marker='o', label='body joint', color = 'black')
            points_2 = ax.scatter(column(each_frame[30:32], 0), column(each_frame[30:32], 1), column(each_frame[30:32], 2), cmap='jet', marker='o', label='body joint', color = 'blue')
            points_3 = ax.scatter(column(each_frame[32:34], 0), column(each_frame[32:34], 1), column(each_frame[32:34], 2), cmap='jet', marker='o', label='body joint', color = 'red')
            
            # ax.scatter(column(each_frame, 0), column(each_frame, 1), column(each_frame, 2), cmap='jet', marker='o', label='body joint')
            # ax.legend()
            # print("+++")
            
            parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                        [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                        [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                        [8, 18], [8, 20], [13, 21], [13, 19],
                        [5, 20], [5, 21], [7, 18], [7, 19],
                        [18, 19], [19, 21], [21, 20], [20, 18], #waist
                        [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                        [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                        [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                        [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                        [30, 31], [32, 33],  #instrument
                        # [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
            lines = []
            # draw line
            
            # lines = [ax.plot([each_frame[vs][0], each_frame[ve][0]],
            #                  [each_frame[vs][1], each_frame[ve][1]],
            #                  [each_frame[vs][2], each_frame[ve][2]]) for (vs, ve) in parents]
            line_num = len(parents)
            for idx, each_line in enumerate(parents):
                vec_start = each_frame[each_line[0]]
                vec_end = each_frame[each_line[1]]
                # print(vec_start)
                # print(vec_end)
                line_color = "black"
                if idx == line_num-2:
                    line_color = "blue"
                if idx == line_num-1:
                    line_color = "red"
                # ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]])
                
                temp, = ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]], color=line_color)
                lines.append(temp)

            # ax.figure.savefig('./test_pic/pic' + str(frame_index) + '.png', dpi=100, bbox_inches = 'tight')

            # ims.append([points])
            # image_frame = [points].extend(lines)
            ims.append([points]+[points_2]+[points_3]+lines) #TODO: try extend

            # plt.cla()
            # print("+++")

    anim = matplotlib.animation.ArtistAnimation(fig, ims, interval=1000/fps)

    if output.endswith('.mp4'):
        FFwriter = matplotlib.animation.FFMpegWriter(fps=fps, extra_args=['-vcodec', 'libx264'])
        anim.save(output, writer=FFwriter)
    elif output.endswith('.gif'):
        anim.save(output, fps=fps, dpi=100, writer='imagemagick')
    else:
        raise ValueError('Unsupported output format (only .mp4 and .gif are supported)')

In [ ]:
def plot(audio_path, plot_path, prediction, sample_time, fps, name=""): #audio_path, plot_path, 
    # render_animation(fps, output='new_temp.mp4', azim=75, prediction=prediction)
    test_render_animation(fps, output='new_temp_' + name + '.mp4', azim=75, prediction=prediction)

    # # #merge with wav
    input_video = ffmpeg.input('new_temp_' + name + '.mp4')
    fluid_syn = FluidSynth()
    fluid_syn.midi_to_audio(audio_path, './output' + name + '.wav')
    input_audio = ffmpeg.input('./output' + name + '.wav')
    # output = ffmpeg.output(video, audio, plot_path, vcodec='copy', acodec='aac', strict='experimental')
    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(plot_path).run()
    # os.remove('new_temp_' + name + '.mp4')

In [ ]:
model.eval()

full_prediction = pd.DataFrame()
num_count = 0
# read midi
# test_dataloader = get_dataloader(test_datapath, batch_size=1)
for test_batch in test_data_list:
    with torch.no_grad():
        first_target = torch.zeros(test_batch.shape[0],102)
        print(first_target.shape)
        test_input = test_batch[None, :]
        test_target = first_target[None, :]
        print("test_input", test_input.shape)
        print("test_target", test_target.shape)
        prediction = predict(model, test_input, test_target, device)
        
        # print(prediction.shape)
        
        prediction  = prediction[:, :, :102]
        print("prediction.shape", prediction.shape)
        
        # full_prediction.append(prediction)
        full_prediction = pd.DataFrame(prediction[0])
        print("full_prediction", full_prediction.shape)
        
        # prev_prediction = prediction[0][:-1][None, :]
        # print(prev_prediction.shape)
        
        Row_list_prediction =[]
        
        filecode = test_music_list[num_count]
    
        # Iterate over each row
        for index, rows in full_prediction.iterrows():
            #fill nan
            rows = rows.fillna(0)
            # Create list for the current row
            my_list = rows.values.tolist()
            # print(my_list)
            
            my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
            # append the list to the final list
            Row_list_prediction.append(my_list_per3)
        
        # print(len(Row_list_prediction), len(Row_list_prediction[0]),len(Row_list_prediction[0][0]))
        plot(test_datapath + test_music_list[num_count] + ".mid", "./video_" + filecode + "_test_predict.mp4", Row_list_prediction[:800], None, 40, filecode) #ow_list[0:900]
        # print("prediction.shape", prediction.shape)

        # formated_motion = prediction_format(full_prediction)
        # # # plot(formated_motion)
        # audio_path = test_music_list[num_count][0]
        # output_path = "test_output_" + filecode + ".mp4"
        # plot(formated_motion, audio_path, output_path, None, 10, filecode)
        num_count += 1

# model.train()

torch.Size([8171, 102])
test_input (1, 8171, 128)
test_target torch.Size([1, 8171, 102])
torch.Size([1, 8171, 102])


prediction.shape (1, 8171, 102)
full_prediction (8171, 102)
81600
limit 800
[[[ 0.05740781  0.12252665  1.09301207]
  [-0.00336951  0.08391601  1.10333154]
  [ 0.10889702  0.06092275  1.10886035]
  ...
  [ 0.02839183  0.08412046  0.96484718]
  [-0.18195246  0.02322661  1.09274397]
  [ 0.10795274  0.23367205  0.80819092]]

 [[ 0.05568753  0.12799329  1.09553376]
  [-0.00352431  0.09023183  1.1088855 ]
  [ 0.10918399  0.06575996  1.11284707]
  ...
  [ 0.01981038  0.08547838  0.96769533]
  [-0.17028056  0.07761087  1.10112665]
  [ 0.0942099   0.19009569  0.79278479]]

 [[ 0.06520741  0.12985748  1.1017457 ]
  [ 0.00260067  0.09447531  1.11584971]
  [ 0.1144044   0.06379886  1.11971531]
  ...
  [ 0.02463806  0.08471523  0.97462485]
  [-0.16447534  0.09507114  1.13074396]
  [ 0.08869245  0.16840369  0.79464409]]

 ...

 [[ 0.01725622  0.09704666  1.10023091]
  [-0.04298529  0.05736584  1.10204468]
  [ 0.06647322  0.03371117  1.10769663]
  ...
  [-0.01686242  0.08000688  0.9665204 ]
  [-0.12

fluidsynth: panic: An error occurred while reading from stdin.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file './outputvs1-1ada.wav'..


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

...

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

torch.Size([7897, 102])
test_input (1, 7897, 128)
test_target torch.Size([1, 7897, 102])
torch.Size([1, 7897, 102])
prediction.shape (1, 7897, 102)
full_prediction (7897, 102)
81600
limit 800
[[[ 0.05286872  0.10052781  1.10362885]
  [-0.01084831  0.06210627  1.10957465]
  [ 0.10044288  0.03420502  1.10794959]
  ...
  [ 0.01440732  0.08105334  0.97696838]
  [-0.10480201  0.11279461  1.07885358]
  [ 0.10670544  0.18397386  0.73580751]]

 [[ 0.04800601  0.10273877  1.10052619]
  [-0.01423963  0.06329025  1.10705194]
  [ 0.09660735  0.03814729  1.1049485 ]
  ...
  [ 0.0106838   0.0827593   0.9723551 ]
  [-0.1183335   0.10224623  1.07977036]
  [ 0.09897475  0.19115892  0.74277947]]

 [[ 0.04656763  0.10538645  1.10035644]
  [-0.01497868  0.06566209  1.10791824]
  [ 0.09685724  0.0421206   1.10527883]
  ...
  [ 0.00944012  0.08477984  0.97294674]
  [-0.1279929   0.09885045  1.08560524]
  [ 0.09278813  0.19166553  0.74470249]]

 ...

 [[ 0.06096807  0.11595962  1.09984294]
  [-0.00324127  0.

fluidsynth: panic: An error occurred while reading from stdin.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file './outputvs1-4prs.wav'..


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55